In [1]:
# import libraries
import os

import numpy as nps
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import mlflow
from pprint import pprint
from minio import Minio
#mlflow.xgboost.autolog()

from helpers import read_data, read_data_from_minio

In [2]:
# specify path to the local data directory 
data_path = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/data/"
data_path

'/Users/abdessamadbaahmed/Desktop/livrable_mp_data/data/'

In [3]:
# initialize minioClient with an endpoint and access/secret keys.
minio_client = Minio('20.224.70.229:9000',
                    access_key='abdessamadbaahmed',
                    secret_key='baahmedabdessamad', secure=False)

# list all buckets
buckets = minio_client.list_buckets()
buckets

[Bucket('nba-investment-data')]

In [4]:
# import data
try:
    train = read_data_from_minio(minio_client, "nba-investment-data", "nba_logreg_processed_train.csv")
    display(train.head())
except Exception:
    train = read_data(f"{data_path}nba_logreg_processed_train.csv")
    train.head()   

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,71.0,10.1,3.2,1.3,3.2,39.6,0.1,0.5,21.9,0.6,0.7,83.7,0.1,0.9,0.9,1.7,0.5,0.1,0.7,1
1,78.0,27.8,8.5,3.6,7.4,48.5,0.5,1.6,32.8,0.8,1.1,74.1,1.2,5.0,6.2,1.2,1.0,1.4,0.7,1
2,52.0,15.4,4.5,1.8,3.6,49.5,0.0,0.0,0.0,1.0,1.4,70.3,1.2,2.1,3.3,0.3,0.2,0.6,1.0,1
3,82.0,19.6,7.4,3.1,7.3,41.9,0.0,0.3,4.3,1.2,1.5,80.6,0.6,1.4,2.0,3.9,1.0,0.3,1.7,1
4,75.0,33.5,11.5,4.4,9.5,45.9,0.1,0.2,35.7,2.7,3.8,70.1,2.5,4.9,7.4,1.0,0.8,1.4,1.4,1


In [5]:
# define features and target
X_train = train.drop('TARGET_5Yrs', axis=1)
y_train = train['TARGET_5Yrs']

In [6]:
# setting the experiment 
mlflow.set_tracking_uri("http://20.224.70.229:5000/")
mlflow.set_experiment("nba-investment-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1673524780458, experiment_id='1', last_update_time=1673524780458, lifecycle_stage='active', name='nba-investment-experiment', tags={}>

In [7]:
# initialize mlflow client
client = mlflow.tracking.MlflowClient()

# Search for experiments
experiments = client.search_experiments()

# Print the experiment ID and name for each experiment
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Experiment name: {experiment.name}")

Experiment ID: 1
Experiment name: nba-investment-experiment


In [8]:
# split the training data into train and validation sets using stratified sampling so that we can preserve the same distribution of the target variable in the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [9]:
# Convert the data to DMatrix objects
train_dmatrix = xgb.DMatrix(X_train, label=y_train)
validation_dmatrix = xgb.DMatrix(X_val, label=y_val)

# Define the objective function for the hyperparameter optimization
def objective(params):
    with mlflow.start_run():
        # Set the model and the search space in the run metadata
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag("holdout_set", "validation set")
        mlflow.log_params(params)

        # Train the XGBoost model using the specified hyperparameters
        booster = xgb.train(
            params=params, # Hyperparameters
            dtrain=train_dmatrix, # Training data
            num_boost_round=1000, # Train for 1000 rounds
            evals=[(validation_dmatrix, 'validation')], # Evaluate on the validation data at each iteration of training 
            early_stopping_rounds=50 # Stop training if the validation score does not improve for 50 rounds
        )
        
        # Make predictions on the validation data
        y_pred = booster.predict(validation_dmatrix).round()
        
        # Calculate the evaluation scores
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        
        # Log the evaluation scores to MLFlow
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

    return {'loss': 1 - f1, 'status': STATUS_OK} # Minimize the negative F1 score

# Define the search space for the hyperparameters
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 200, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'binary:logistic',
    'seed': 42
}


# Perform the hyperparameter optimization using the Tree Parzen Estimator algorithm
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-logloss:0.64414                        
[1]	validation-logloss:0.61446                        
[2]	validation-logloss:0.61225                        
[3]	validation-logloss:0.60720                        
[4]	validation-logloss:0.61857                        
[5]	validation-logloss:0.61431                        
[6]	validation-logloss:0.61507                        
[7]	validation-logloss:0.62557                        
[8]	validation-logloss:0.64640                        
[9]	validation-logloss:0.65651                        
[10]	validation-logloss:0.64616                       
[11]	validation-logloss:0.65831                       
[12]	validation-logloss:0.64456                       
[13]	validation-logloss:0.64121                       
[14]	validation-logloss:0.63220                       
[15]	validation-logloss:0.63781                       
[16]	validation-logloss:0.64057                       
[17]	validation-logloss:0.63963                       
[18]	valid

In [10]:
best_result["max_depth"] = int(best_result["max_depth"])
best_result["objective"] = "binary:logistic"
best_result["seed"] = 42
best_result

{'learning_rate': 0.6998112100554166,
 'max_depth': 47,
 'min_child_weight': 10.547500610440071,
 'reg_alpha': 0.006755667199912489,
 'reg_lambda': 0.003970871219886506,
 'objective': 'binary:logistic',
 'seed': 42}

In [11]:
with mlflow.start_run():
    # Log the best hyperparameters to MLFlow
    mlflow.log_params(best_result)
    mlflow.set_tag("model", "xgboost")
    mlflow.set_tag("holdout_set", "validation set")

    # Train the XGBoost model using the specified hyperparameters
    booster = xgb.train(
        params=best_result, # Hyperparameters
        dtrain=train_dmatrix, # Training data
        num_boost_round=1000, # Train for 1000 rounds
        evals=[(validation_dmatrix, 'validation')], # Evaluate on the validation data at each iteration of training
        early_stopping_rounds=50 # Stop training if the validation score does not improve for 50 rounds
    )
    
    # Make predictions on the validation data
    y_pred = booster.predict(validation_dmatrix).round()
    
    # Calculate the evaluation scores
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    # Log the evaluation scores to MLFlow
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
        
    # mlflow.log_artifact("mymodel", artifact_path="model")  
    # mlflow.framework.log_model(model_object, artifact_path="model")  
    mlflow.xgboost.log_model(booster, "model")    

[0]	validation-logloss:0.61323
[1]	validation-logloss:0.59713
[2]	validation-logloss:0.61890
[3]	validation-logloss:0.61747
[4]	validation-logloss:0.61713
[5]	validation-logloss:0.61323
[6]	validation-logloss:0.62045
[7]	validation-logloss:0.61140
[8]	validation-logloss:0.61367
[9]	validation-logloss:0.60665
[10]	validation-logloss:0.60615
[11]	validation-logloss:0.62465
[12]	validation-logloss:0.63433
[13]	validation-logloss:0.63399
[14]	validation-logloss:0.64329
[15]	validation-logloss:0.65004
[16]	validation-logloss:0.65257
[17]	validation-logloss:0.64570
[18]	validation-logloss:0.63942
[19]	validation-logloss:0.63503
[20]	validation-logloss:0.62602
[21]	validation-logloss:0.63089
[22]	validation-logloss:0.63818
[23]	validation-logloss:0.65852
[24]	validation-logloss:0.66250
[25]	validation-logloss:0.65913
[26]	validation-logloss:0.66408
[27]	validation-logloss:0.65845
[28]	validation-logloss:0.66345
[29]	validation-logloss:0.66122
[30]	validation-logloss:0.65977
[31]	validation-lo

/Users/abdessamadbaahmed/opt/anaconda3/envs/mlops_project/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:
for mv in client.search_model_versions("name='nba-investment-model-f1'"):
    pprint(dict(mv), indent=4)

{   'creation_timestamp': 1673879935564,
    'current_stage': 'Production',
    'description': '',
    'last_updated_timestamp': 1674078333887,
    'name': 'nba-investment-model-f1',
    'run_id': '71331902263b445cb6b7789c4334cd42',
    'run_link': '',
    'source': 'mlflow-artifacts:/1/71331902263b445cb6b7789c4334cd42/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}
{   'creation_timestamp': 1674078328843,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1674078328843,
    'name': 'nba-investment-model-f1',
    'run_id': '7ce4b20ae2324f839a8fd58fb809db68',
    'run_link': '',
    'source': 'mlflow-artifacts:/1/7ce4b20ae2324f839a8fd58fb809db68/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '10'}
{   'creation_timestamp': 1674072381684,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestam

In [12]:
# retrieve a model from the registry
#logged_model = 'runs:/25d40fe0311a4314b7bfa6cfd9669a8e/model'

# Load model as a PyFuncModel.
#loaded_model = mlflow.pyfunc.load_model(logged_model)

# convert the loaded model to xgboost model
#xgb_model = mlflow.xgboost.load_model(logged_model)

## cross-validation version - very intensive in computation

In [8]:
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np


# Define the objective function for the hyperparameter optimization
def objective(params, X_train, y_train, k=5):

    X_train = X_train.values
    y_train = y_train.values

    # Convert the data to DMatrix objects
    train_dmatrix = xgb.DMatrix(X_train, label=y_train)

    with mlflow.start_run():
        # Set the model and the search space in the run metadata
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        # Create a KFold object for cross-validation
        kf = KFold(n_splits=k, shuffle=True, random_state=42)

        # Initialize the evaluation scores
        accuracy = []
        precision = []
        recall = []
        f1 = []

        # Iterate over the folds
        for train_index, val_index in kf.split(X_train):
            # Split the data into training and validation sets
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

            # Convert the data to DMatrix objects
            train_fold_dmatrix = xgb.DMatrix(X_train_fold, label=y_train_fold)
            val_fold_dmatrix = xgb.DMatrix(X_val_fold, label=y_val_fold)

            # Train the XGBoost model using the specified hyperparameters
            booster = xgb.train(
                params=params,
                dtrain=train_fold_dmatrix,
                num_boost_round=1000,
                evals=[(val_fold_dmatrix, 'validation')],
                early_stopping_rounds=50
            ) 
            
            # Make predictions on the validation data
            y_pred = booster.predict(val_fold_dmatrix).round()

            # Calculate the evaluation scores for the fold
            accuracy.append(accuracy_score(y_val_fold, y_pred))
            precision.append(precision_score(y_val_fold, y_pred))
            recall.append(recall_score(y_val_fold, y_pred))
            f1.append(f1_score(y_val_fold, y_pred))

        # Calculate the mean evaluation scores over all the folds
        mean_accuracy = np.mean(accuracy)
        mean_precision = np.mean(precision)
        mean_recall = np.mean(recall)
        mean_f1 = np.mean(f1)
        
        # Log the evaluation scores to MLFlow
        mlflow.log_metric("accuracy", mean_accuracy)
        mlflow.log_metric("precision", mean_precision)
        mlflow.log_metric("recall", mean_recall)
        mlflow.log_metric("f1_score", mean_f1)

    return {'loss': 1 - mean_f1, 'status': STATUS_OK}

In [9]:
# Define the search space for the hyperparameters
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'binary:logistic',
    'seed': 42
}

In [ ]:
# Perform the hyperparameter optimization using the Tree Parzen Estimator algorithm
def objective_cv(params):
    return objective(k=5, X_train=X_train, y_train=y_train, params=params)

best_result = fmin(
    fn=objective_cv,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)